In [ ]:
# ЗАПУСТИТЕ ЭТОТ КОД, ЕСЛИ ХОТИТЕ СКАЧАТЬ НАБОР ДАННЫХ ПО ССЫЛКЕ KAGGLE

# Загрузка файла по ссылке Kaggle в сессионный набор файлов блокнота.
# При этом необходимо будет ввести свой логин и токен для подключения к Kaggle
# (для его получения на странице своего аккаунта в Kaggle https://www.kaggle.com/settings/account нужно нажать кнопку Create New Token
# и в поле Key вставить полученное в файле json значение)

!pip install opendatasets --upgrade --quiet
import opendatasets as od
import pandas as pd
od.download(
   "https://www.kaggle.com/datasets/yasserh/titanic-dataset", force=True)
df = pd.read_csv('./titanic-dataset/Titanic-Dataset.csv')
print('ИСХОДНЫЙ НАБОР ДАННЫХ:')
display(df)

In [43]:
# ЗАПУСТИТЕ ЭТОТ КОД, ЕСЛИ ХОТИТЕ СКАЧАТЬ НАБОР ДАННЫХ, РАЗМЕЩЕННЫЙ НА GOOGLE DISK

import pandas as pd
url = 'https://drive.google.com/file/d/1NLS6-YCjx9ck-E76rERyUWqnIF-uBfC2/view?usp=sharing'
dataset_url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(dataset_url)
df.set_index('PassengerId', inplace = True)
print('ИСХОДНЫЙ НАБОР ДАННЫХ:')
display(df)

ИСХОДНЫЙ НАБОР ДАННЫХ:


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


In [ ]:
# !!!СНАЧАЛА ВЫПОЛНИТЕ СКАЧИВАНИЕ НАБОРА ДАННЫХ, ЗАПУСТИВ ПЕРВЫЙ ИЛИ ВТОРОЙ БЛОК КОДА
import numpy as np

# Посчитаем и выведем количество пустых ячеек в исходном наборе данных
count_of_null_original = df.isna().sum().sum()
print()
print(f'Количество пустых ячеек в исходном наборе = {count_of_null_original}')

# Скопируем первоначальный набор данных и будем изменять его копию
df_without_null = df.copy()

# Пустые ячейки в столбце возраста заполним средним значением, а затем переведем в целое число
df_without_null['Age'].fillna(df_without_null['Age'].mean(), inplace = True)
df_without_null['Age'] = df_without_null['Age'].apply(np.int32)

# Пустые ячейки в столбце с номером каюты заполним рандомными значениями, исходя из того, что палуб было 6: "A", "B", "C", "D", "F", "G"
# и самое маленькое количество кают было 34. То есть составим список рандомных значений номеров кают с учетом ограничения по максимальному номеру (34) для верхней палубы
cabin_class_list = ["A", "B", "C", "D", "F", "G"]
cabin_num_list = list(range(1, 35))
cabin_choise_list = [(x + str(y)) for x in cabin_class_list for y in cabin_num_list]
df_without_null['Cabin'].fillna(pd.Series(np.random.choice(cabin_choise_list, size = len(df_without_null.index) + 1)), inplace = True)

# Пустые ячейки в столбце порта посадки заполним значением из предыдущей строки
df_without_null['Embarked'].fillna(method="pad", inplace = True)

# Посчитаем и выведем количество пустых ячеек в измененном наборе данных (должно быть 0)
count_of_null_changed = df_without_null.isna().sum().sum()
print()
print(f'Количество пустых ячеек в измененном наборе = {count_of_null_changed}')

print()
print('НАБОР ДАННЫХ С ЗАПОЛНЕННЫМИ ПУСТЫМИ ЯЧЕЙКАМИ:')
display(df_without_null)

# Выведем информацию по каждому признаку: минимальное, максимальное значение (для числовых признаков) и количество уникальных значений
df_without_null_info = df_without_null.agg(
    {
        "Pclass": ["min", "max", "nunique"],
        "Name": ['nunique'],
        "Sex": ["nunique"],
        "Age": ["min", "max", "nunique"],
        "SibSp": ["min", "max", "nunique"],
        "Parch": ["min", "max", "nunique"],
        "Ticket": ["nunique"],
        "Fare": ["min", "max", "nunique"],
        "Cabin": ["nunique"],
        "Embarked": ["nunique"]
    }
)
print()
print('ИНФОРМАЦИЯ ПО ДАННЫМ:')
display(df_without_null_info)

# Проведем аналитику: вычислим процент выживания в зависимости от различных признаков. В качестве признаков взяла пол, возраст, класс круиза, номер каюты, палубу.
# Для группировке по палубе добавила столбец, в котором выделена буква из номера каюты, обозначающая палубу

df_survived_sex = df_without_null[["Sex", "Survived"]].groupby("Sex").apply(lambda x: x['Survived'].sum() / len(x) * 100)
print()
print('ПРОЦЕНТ ВЫЖИВАНИЯ ВНУТРИ ГРУПП КАЖДОГО ПОЛА:')
display(df_survived_sex)

df_survived_age = df_without_null[["Age", "Survived"]].groupby("Age").apply(lambda x: x['Survived'].sum() / len(x) * 100)
print()
print('ПРОЦЕНТ ВЫЖИВАНИЯ В КАЖДОЙ ВОЗРАСТНОЙ ГРУППЕ:')
display(df_survived_age)

df_survived_pclass = df_without_null[["Pclass", "Survived"]].groupby("Pclass").apply(lambda x: x['Survived'].sum() / len(x) * 100)
print()
print('ПРОЦЕНТ ВЫЖИВАНИЯ ВНУТРИ ГРУПП КАЖДОГО КЛАССА КРУИЗА:')
display(df_survived_pclass)

df_without_null['Deck'] = df_without_null['Cabin'].apply(lambda x: x[0])
df_survived_deck = df_without_null[["Deck", "Survived"]].groupby("Deck").apply(lambda x: x['Survived'].sum() / len(x) * 100)
print()
print('ПРОЦЕНТ ВЫЖИВАНИЯ ВНУТРИ ГРУПП КАЖДОЙ ПАЛУБЫ:')
display(df_survived_deck)

# Подведем итоги анализа
resume = {'Признак': ['Пол',
                      'Возраст',
                      'Клас круиза',
                      'Палуба'],
          'Вывод': ['Выживаемость среди женщин значительно выше. Скорее всего, причина в том, что их сажали в шлюпки в первую очередь',
                    'Выживаемость среди детей до 16 лет выше, чем в других возрастных группах (без учета 0 и 100, что связано с малым количеством людей в группе). Опять же их сажали в шлюпки в первую очередь',
                    'Видна прямая зависимость: чем выше класс, тем больше процент выживания. На мой взгляд, у них был проще доступ к шлюпкам, а также, возможно, пассажирам из 1го и 2го классов оказывали приоритет по спасению',
                    'Результат интересен тем, что самый большой процент выживаемости оказался на средней палубе (E), хотя сами шлюпки расположены были на самой верхней палубе. В целом же на высоких палубах процент выживаемости больше, чем на нижних, что понятно.']}
df_resume = pd.DataFrame(resume)
pd.options.display.max_colwidth = 256
print()
print('НЕКОТОРЫЕ ВЫВОДЫ:')
display(df_resume)